In [6]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/jwfree/anaconda3/envs/CLASSIFIER/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [7]:
import numpy as np
from sklearn import metrics
import os

In [8]:
os.listdir("/home/jwfree/rc0_data")

['curvature', 'angle', 'direction', 'area']

In [17]:
import copy
import statistics as s
results_template = {
    "vgg19": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    },
    "alexnet": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    },
    "densenet": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    },
    "resnet": {
        "mlae": [],
        "msqr_error": [],
        "total_good_results": [],
        "total_bad_results": []
    }
}
results = {}
results_total = {}
rc0 = "/home/jwfree/rc0_data"
rc1 = "/home/jwfree/rc1_data"
for cluster in [rc0, rc1]:
    for stim in os.listdir(cluster):
        
        results_total[stim] = {}
        
        results[stim] = copy.deepcopy(results_template)
        stim_path = os.path.join(cluster, stim)

        for model in os.listdir(stim_path):

            model_path = os.path.join(stim_path, model)
            results_file = os.path.join(model_path, "results.npz")

            if os.path.exists(results_file):

                files = os.listdir(model_path)
                key = None
                if "vgg19" in files:
                    key = "vgg19"
                elif "alexnet" in files:
                    key = "alexnet"
                elif "densenet" in files:
                    key = "densenet"
                elif "resnet" in files:
                    key = "resnet"
                else:
                    continue

                data = np.load(results_file)
                

                
                y_test  = data["labels"]
                l_min = labels.min()
                l_max = labels.max()
                
                y_test  = y_test / (l_max - l_min)
                
                y_pred = data["results"]
                y_pred = y_pred / (l_max - l_min)
                
                mlae = np.log2(metrics.mean_absolute_error(y_pred*100, y_test*100)+.125)

                results[stim][key]["mlae"].append(mlae.tolist())
                results[stim][key]["msqr_error"].append(data["msqr_error"].tolist())
                results[stim][key]["total_bad_results"].append(len(data["bad_result_ids"]))
                results[stim][key]["total_good_results"].append(len(data["results"]))

        for model in results_template.keys():
            results_total[stim][model] = {}
            results_total[stim][model]["runs"] = len(results[stim][model]["msqr_error"])
            if results_total[stim][model]["runs"] != 0:
                results_total[stim][model]["mlae"] = results[stim][model]["mlae"]
                results_total[stim][model]["msqr_error"] = results[stim][model]["msqr_error"]
                results_total[stim][model]["total_bad_results"] = results[stim][model]["total_bad_results"]
                results_total[stim][model]["total_good_results"] = results[stim][model]["total_good_results"]
    
            
            

In [18]:
import yaml
print(yaml.dump(results_total))
with open("results.yml", "w") as file:
    yaml.dump(results_total, file)

angle:
  alexnet:
    mlae:
    - 8.412265332412948
    - 8.462266006939426
    - 8.334140615408794
    - 8.418515691453601
    - 8.584198506913253
    - 8.328612019392876
    msqr_error:
    - 22.100527501854984
    - 25.525067670461056
    - 20.29270690278037
    - 22.99079430343363
    - 29.70589682965315
    - 20.108964262244953
    runs: 6
    total_bad_results:
    - 5
    - 282
    - 96
    - 1
    - 33
    - 2
    total_good_results:
    - 19995
    - 19718
    - 19904
    - 19999
    - 19967
    - 19998
  densenet:
    mlae:
    - 8.511468312517724
    - 8.472601592801595
    - 8.492086755351528
    - 8.611012717494695
    - 8.626921727525925
    - 8.788762861667639
    msqr_error:
    - 26.34437966646048
    - 24.87276380620484
    - 25.64406882578905
    - 30.915325941487392
    - 30.820960445054496
    - 37.59466776207357
    runs: 6
    total_bad_results:
    - 0
    - 65
    - 0
    - 21
    - 43
    - 17
    total_good_results:
    - 20000
    - 19935
    - 20000
    - 1